<a href="https://colab.research.google.com/github/siwei-li/LC-graph/blob/master/LCScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time,csv,sys,pytz,os
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select,WebDriverWait

import numpy as np
from datetime import date,datetime,timedelta

In [2]:
# refer following discussion
# https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path
# install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# !pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# # open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [6]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%load_ext dotenv
%dotenv

In [8]:
!pip install py2neo -q

In [4]:
from py2neo import Graph, Node, Relationship

g = Graph(f"neo4j+s://{os.getenv('AURA_DOMAIN')}.databases.neo4j.io", auth=("neo4j", os.getenv('AURA_PWD')), routing=True)


In [5]:
print(g.run("MATCH (n) RETURN n"))

 n                                                                          
----------------------------------------------------------------------------
 (_0:Tag {title: 'Array', uri: 'https://leetcode.com/tag/array'})           
 (_1:Tag {title: 'String', uri: 'https://leetcode.com/tag/string'})         
 (_2:Tag {title: 'Hash Table', uri: 'https://leetcode.com/tag/hash-table'}) 



In [40]:
# res = g.run("MATCH (n)-[r]-(n2) RETURN r");
# for r in res:
#     print(r)

In [59]:
def login():
    url = 'https://leetcode.com/accounts/login'
    driver.get(url)
    WebDriverWait(driver, timeout=15).until_not(lambda x: x.find_element(By.CLASS_NAME, "spinner").is_displayed())
    try:
        emailInput = driver.find_element(By.ID, 'id_login')
        passwordInput = driver.find_element(By.ID,'id_password')
        loginButton = driver.find_element(By.ID,'signin_btn')
        emailInput.send_keys(os.getenv('LC_USERNAME'))
        passwordInput.send_keys(os.getenv('LC_PWD'))
        loginButton.click()
    except:
        pass
    finally:
        WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.CLASS_NAME,"trending-topic-container").is_displayed())

In [6]:
driver = webdriver.Chrome('chromedriver', options=options)
# login()

WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 127


In [58]:
def problemsOnPage(page=1):
    url = 'https://leetcode.com/problemset/all/'
    if page!=1:
      url = f"https://leetcode.com/problemset/all/?page={page}"
    driver.get(url)

    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@role = 'table']").is_displayed())

    script = '''
const rows = document.querySelectorAll("[role=rowgroup]")[0].children;
const problems = [];
for (const row of rows) {
  const problem = [], cols = row.children;

  const link = cols[1].querySelector('a').href;

  problem.push(link);
  problem.push(cols[1].innerText.trim());
  problem.push(cols[3].innerText.trim());
  problem.push(cols[4].innerText.trim());

  problems.push(problem);
}
return problems
    '''
    problemsData = driver.execute_script(script)
#     problems = pd.DataFrame(problemsData, columns = ['Link','Title','Acceptance','Difficulty'])
    if page==1:
        return problems[1:]
    return problems

In [47]:
def create_node_from_row(uri, title, acceptance, difficulty):
  n = Node("Problem", uri=uri, title=title, acceptance=acceptance, difficulty=difficulty)
  create_tx.create(n)

p1.apply(lambda r: create_node_from_row(r[0],r[1],r[2], r[3]), axis=1)

g.commit()

In [55]:
def get_problem_info(url):

    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'description__24sA']").is_displayed())

    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script)
    
    simsData = []
    

    return list(tagsData), list(simsData)

In [56]:
get_problem_info('https://leetcode.com/problems/two-sum/')

MaxRetryError: HTTPConnectionPool(host='localhost', port=57967): Max retries exceeded with url: /session/3b0e5fb521a0c8e6bc9ba4e4746384e1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fabe9948370>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
def find_and_commit(tags, sims):
    

In [52]:
PAGE_CNT = 49

In [ ]:
for page_num in range(PAGE_CNT):
    print(f"Page {page_num} started..")
    probs = problemsOnPage(page=1)
    print("Got problem list...")
    df = pd.DataFrame(probs)
    df.to_csv(f'problem_p{page_num}.csv')
    
    for prob in probs:
        url = prob[0]
        tags, sims = get_problem_info(url)
        find_and_commit(tags, sims)
    

In [46]:
p1

,Link,Title,Acceptance,Difficulty
0,https://leetcode.com/problems/continuous-subar...,523. Continuous Subarray Sum,27.8%,Medium
1,https://leetcode.com/problems/two-sum/,1. Two Sum,49.1%,Easy
2,https://leetcode.com/problems/add-two-numbers/,2. Add Two Numbers,39.7%,Medium
3,https://leetcode.com/problems/longest-substrin...,3. Longest Substring Without Repeating Characters,33.8%,Medium
4,https://leetcode.com/problems/median-of-two-so...,4. Median of Two Sorted Arrays,35.2%,Hard
5,https://leetcode.com/problems/longest-palindro...,5. Longest Palindromic Substring,32.4%,Medium
6,https://leetcode.com/problems/zigzag-conversion/,6. Zigzag Conversion,43.0%,Medium
7,https://leetcode.com/problems/reverse-integer/,7. Reverse Integer,27.2%,Medium
8,https://leetcode.com/problems/string-to-intege...,8. String to Integer (atoi),16.6%,Medium
9,https://leetcode.com/problems/palindrome-number/,9. Palindrome Number,52.8%,Easy


In [41]:
create_tx = g.begin()

In [16]:
def get_tags():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)

    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'relative flex mb-1']").is_displayed())

    script = '''
const data = document.querySelector("div[class='relative flex mb-1']").firstChild.children;
const tags = [];
for (const item of data) {
  const tag = [];

  const link = item.querySelector('a').href;

  tag.push(link);
  tag.push(item.querySelector('span').innerText);

  tags.push(tag);
}
return tags
    '''
    tagsData = driver.execute_script(script)
   
    return list(tagsData)

In [17]:
tags = get_tags()

In [22]:
tgs = tags
tgs

[['https://leetcode.com/tag/array', 'Array'],
 ['https://leetcode.com/tag/string', 'String'],
 ['https://leetcode.com/tag/hash-table', 'Hash Table'],
 ['https://leetcode.com/tag/dynamic-programming', 'Dynamic Programming'],
 ['https://leetcode.com/tag/math', 'Math'],
 ['https://leetcode.com/tag/sorting', 'Sorting'],
 ['https://leetcode.com/tag/greedy', 'Greedy'],
 ['https://leetcode.com/tag/depth-first-search', 'Depth-First Search'],
 ['https://leetcode.com/tag/database', 'Database'],
 ['https://leetcode.com/tag/breadth-first-search', 'Breadth-First Search'],
 ['https://leetcode.com/tag/tree', 'Tree'],
 ['https://leetcode.com/tag/binary-search', 'Binary Search'],
 ['https://leetcode.com/tag/matrix', 'Matrix'],
 ['https://leetcode.com/tag/binary-tree', 'Binary Tree'],
 ['https://leetcode.com/tag/two-pointers', 'Two Pointers'],
 ['https://leetcode.com/tag/bit-manipulation', 'Bit Manipulation'],
 ['https://leetcode.com/tag/stack', 'Stack'],
 ['https://leetcode.com/tag/heap-priority-queue'

In [24]:
tags = pd.DataFrame(tgs)
tags

,0,1
0,https://leetcode.com/tag/array,Array
1,https://leetcode.com/tag/string,String
2,https://leetcode.com/tag/hash-table,Hash Table
3,https://leetcode.com/tag/dynamic-programming,Dynamic Programming
4,https://leetcode.com/tag/math,Math
...,...,...
66,https://leetcode.com/tag/eulerian-circuit,Eulerian Circuit
67,https://leetcode.com/tag/radix-sort,Radix Sort
68,https://leetcode.com/tag/strongly-connected-co...,Strongly Connected Component
69,https://leetcode.com/tag/rejection-sampling,Rejection Sampling


In [51]:
tags.to_csv('tags.csv')

In [50]:
len(tgs)

71

In [27]:
tags.iloc[0,:]

0    https://leetcode.com/tag/array
1                             Array
Name: 0, dtype: object

In [31]:
tgs[0]

['https://leetcode.com/tag/array', 'Array']

In [41]:
create_tx = g.begin()

In [ ]:
n = Node("Tag", uri='https://leetcode.com/tag/array', title='Array')
create_tx.create(n)

In [43]:
g.commit()

/tmp/ipykernel_109057/2607430192.py:1: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


In [46]:
def create_tag_from_row(uri, title):
  n = Node("Tag", uri=uri, title=title)
  create_tx.create(n)

tags.iloc[1:,:].apply(lambda r: create_tag_from_row(r[0],r[1]), axis=1)

g.commit()

TypeError: Cannot run query in closed transaction

In [48]:
create_tx = g.begin()

In [49]:
for uri, title in tgs[1:]:
    n = Node("Tag", uri=uri, title=title)
    create_tx.create(n)

create_tx.commit()

/tmp/ipykernel_109057/2121119833.py:5: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


In [ ]:
driver.quit()



---



In [ ]:
def logTime(startTime, currentTime):
    endTime = time.time()
    currentTime = currentTime or endTime
    hours, rem = divmod(endTime-currentTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("[Duration:{:0>2}:{:0>2}:{:05.2f}, ".format(int(hours),int(minutes),int(seconds)), end='')
    hours, rem = divmod(endTime-startTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Total:{:0>2}:{:0>2}:{:05.2f}] ".format(int(hours),int(minutes),seconds), end='')
    return endTime

In [ ]:
# def export(df, name, path = ''):
#     now = lambda: datetime.now().astimezone().isoformat(timespec='seconds').replace('-','').replace(':','')
#     fullPath = 'Scraped-Data/' + path
#     Path(fullPath).mkdir(parents=True, exist_ok=True)
#     df.to_csv(fullPath + '/' + name + '-' + now() + '.csv')

In [ ]:
def getProblemsByTitle():
    return allProblems.reset_index().set_index('Title')

In [ ]:
def openAndExportAllCompanies():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_tag_name("select").is_displayed())
    driver.find_element_by_css_selector('#expand-company .btn').click()
    script = '''
const buttons = document.getElementById('current-company-tags').children;
const companies = [];
for (const button of buttons) {
  const company = [];
  company.push(button.firstElementChild.innerText.trim());
  company.push(button.firstElementChild.nextElementSibling.innerText.trim());
  company.push(button.href);
  companies.push(company);
}
return companies
    '''
    companiesData = driver.execute_script(script)
    companies = pd.DataFrame(companiesData, columns = ['Name','Count','Link'])
    companies.index = companies.index + 1
    companies.index.name = 'Serial'
    export(companies, 'Companies')
    print("Exported %d Companies (100%%)" % len(companies))
    return companies

In [ ]:
def getFirstOption():
    driver.find_element_by_id('react-select-2--value-item').click()
    firstOption = driver.execute_script("return (4-document.querySelectorAll('#react-select-2--list div[role=option]').length);")
    driver.find_element_by_id('react-select-2--value-item').click()
    return firstOption

In [ ]:
def selectOption(n):
    driver.find_element_by_id('react-select-2--value-item').click()
    driver.find_element_by_id('react-select-2--option-' + str(n)).click()

In [ ]:
def getCompanyProblems():
    script = '''
const rows = document.getElementsByClassName('reactable-data')[0].children;
const problems = [];
for (const row of rows) {
    const problem = [], cols = row.children;
    const tags = Array.from(cols[3].querySelectorAll('a')).map(t => t.innerText.trim()).join(', ');
    const frequency = parseFloat(row.getElementsByClassName('progress-bar')[0].style.width);
    const link = cols[2].querySelector('a').href;
    problem.push(cols[1].innerText.trim());
    problem.push(cols[2].innerText.trim());
    problem.push(tags);
    problem.push(cols[4].innerText.trim());
    problem.push(cols[5].innerText.trim());
    problem.push(frequency);
    problem.push(link);
    problems.push(problem);
}
return problems
    '''
    companyProblemsData = driver.execute_script(script)
    companyProblems = pd.DataFrame(companyProblemsData, columns = ['Id','Title','Tags','Acceptance','Difficulty','Frequency','Link'])
    companyProblems = companyProblems.set_index('Id')
    return companyProblems

In [ ]:
def openAndExportAllCompanyProblems():
    startTime = time.time()
    currentTime = startTime
    optionNames = ['6_months', '1_year', '2_years', 'All_time']
    optionTotalCount = len(optionNames)
    companiesCount = len(allCompanies)
    currentTime = logTime(startTime, currentTime); print("Total Companies: %d" % companiesCount)
    for i in range(companiesCount):
        company = allCompanies.iloc[i]
        companyName = company['Name'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (companyName, i+1,companiesCount,((i+1)*100/companiesCount)))
        driver.get(company['Link'])
        WebDriverWait(driver, timeout=30).until(lambda x: x.find_element_by_id('react-select-2--value-item').is_displayed())
        driver.find_element_by_css_selector('input[type=checkbox]').click()
        driver.find_element_by_css_selector('.reactable-column-header th:nth-child(2)').click()
        firstOption = getFirstOption()
        currentTime = logTime(startTime, currentTime); print("Periods available: %d" % (optionTotalCount- firstOption))
        currentTime = logTime(startTime, currentTime); 
        for currentOption in range(firstOption, optionTotalCount):
            optionName = optionNames[currentOption]
            selectOption(currentOption)
            companyProblems = getCompanyProblems()
            print("%s: %d | " % (optionName, len(companyProblems)), end='')
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Period-wise/' + optionName)
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Name-wise/' + companyName)
        print()
    currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Companies (100%%)" % companiesCount)

In [ ]:
def openAndExportAllInterviews():
    url = 'https://leetcode.com/explore/interview/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_class_name('explore-card').is_displayed())
    script = '''
const cards = Array.from(document.querySelectorAll('.explore-card .premium')).map(e => e.closest('.explore-card'))
const interviews = [];
for (const card of cards) {
  const interview = [];
  interview.push(card.querySelector('.title').innerText.trim());
  interview.push(card.querySelector('.chapter').innerText.trim());
  interview.push(card.querySelector('.item').innerText.trim());
  interview.push(card.querySelector('a').href);
  interviews.push(interview);
}
return interviews;
    '''
    interviewsData = driver.execute_script(script)
    interviews = pd.DataFrame(interviewsData, columns = ['Title','Chapters','Problems','Link'])
    interviews.index = interviews.index + 1
    interviews.index.name = 'Serial'
    export(interviews, 'Interviews')
    print("Exported %d Interviews (100%%)" % len(interviews))
    return interviews

In [ ]:
def joinProblemDetails(problem):
    problem['Id'] = None
    cols = ['Acceptance','Difficulty','Frequency','Link']
    for col in cols:
        problem[col] = None
    try:
        problem['Id'] = allProblemsByTitle.loc[problem['Problem']]['Id']
        for col in cols:
            problem[col] = allProblems.loc[problem['Id']][col]
    except:
        pass
    return problem

In [ ]:
def getInterviewProblems():
    script = '''
const tables = document.getElementsByClassName('table-base');
const problems = [];
for (let i = 1; i <= tables.length; i++) {
  const table = tables[i-1];
  const chapterTitle = table.querySelector('.chapter-title').innerText.trim();
  const items = table.querySelectorAll('*[title="Coding Question"] ~ span');
  for (let j = 1; j <= items.length; j++) {
  const item = items[j-1];
    const itemTitle = item.innerText.trim();
    problems.push([i, chapterTitle, j, itemTitle]);
  }
}
return problems
    '''
    interviewProblemsData = driver.execute_script(script)
    interviewProblems = pd.DataFrame(interviewProblemsData, columns = ['ChapterSerial','Chapter','ProblemSerial','Problem'])
    interviewProblems = interviewProblems.apply(joinProblemDetails, axis=1)
    interviewProblems = interviewProblems.set_index('ChapterSerial')
    return interviewProblems

In [ ]:
def openAndExportAllInterviewProblems():
    startTime = time.time()
    currentTime = startTime
    interviewsCount = len(allInterviews)
    currentTime = logTime(startTime, currentTime); print("Total interviews: %d" % interviewsCount)
    for i in range(interviewsCount):
        interview = allInterviews.iloc[i]
        interviewTitle = interview['Title'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (interviewTitle, i+1,interviewsCount,((i+1)*100/interviewsCount)))
        driver.get(interview['Link'])
        WebDriverWait(driver, timeout=10).until(lambda x: str(len(x.find_elements_by_css_selector('.table-base .table-item'))) == interview['Problems'])
        interviewProblems = getInterviewProblems()
        currentTime = logTime(startTime, currentTime); print("Chapters: %s | Problems: %d" % (interview['Chapters'], len(interviewProblems)))
        export(interviewProblems, interviewTitle, 'Interview_Problems')
        currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Interviews (100%%)" % interviewsCount)

In [ ]:
allProblems = openAndExportAllProblems()

Exported 1579 Problems (100%)


In [ ]:
allProblemsByTitle = getProblemsByTitle()

In [ ]:
allCompanies = openAndExportAllCompanies()

Exported 236 Companies (100%)


In [ ]:
openAndExportAllCompanyProblems()

Duration:00:00:01.00, Total:00:06:14.67] Periods available: 2
[Duration:00:00:00.00, Total:00:06:14.67] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:14.81] Processing GE_Digital (138/236) (58%)
[Duration:00:00:02.00, Total:00:06:17.02] Periods available: 2
[Duration:00:00:00.00, Total:00:06:17.02] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:17.13] Processing infosys (139/236) (58%)
[Duration:00:00:01.00, Total:00:06:19.08] Periods available: 1
[Duration:00:00:00.00, Total:00:06:19.08] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:19.14] Processing Jane_Street (140/236) (59%)
[Duration:00:00:03.00, Total:00:06:22.23] Periods available: 2
[Duration:00:00:00.00, Total:00:06:22.23] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:22.36] Processing Machine_Zone (141/236) (59%)
[Duration:00:00:01.00, Total:00:06:24.31] Periods available: 1
[Duration:00:00:00.00, Total:00:06:24.31] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:24.37] 

In [ ]:
allInterviews = openAndExportAllInterviews()

Exported 10 Interviews (100%)


In [ ]:
openAndExportAllInterviewProblems()

[Duration:00:00:00.00, Total:00:00:00.00] Total interviews: 10
[Duration:00:00:00.00, Total:00:00:00.00] Processing Google_Interview (1/10) (10%)
[Duration:00:00:03.00, Total:00:00:03.89] Chapters: 9 | Problems: 81
[Duration:00:00:00.00, Total:00:00:03.89] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:03.89] Processing Facebook (2/10) (20%)
[Duration:00:00:03.00, Total:00:00:07.58] Chapters: 9 | Problems: 79
[Duration:00:00:00.00, Total:00:00:07.58] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:07.58] Processing Microsoft (3/10) (30%)
[Duration:00:00:03.00, Total:00:00:10.62] Chapters: 8 | Problems: 58
[Duration:00:00:00.00, Total:00:00:10.62] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:10.62] Processing Amazon (4/10) (40%)
[Duration:00:00:03.00, Total:00:00:13.72] Chapters: 8 | Problems: 68
[Duration:00:00:00.00, Total:00:00:13.72] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00

In [ ]:
driver.quit()